# Basic/B2a Example
It is equivalent to the B2a example in Geant4 but re-written with a new more Julia friendly interface. The core of the example is to create an instance of the type `G4JLApplication` with all the essential elements defined (detector and geometry, physics list, primary particles generator, actions, simulated data being accumulated, mapping of logical volumes to sensitive detectors, etc.) 

#### Import the `Geant4` and `Geant4.SystemOfUnits` modules

In [1]:
using Geant4
using Geant4.SystemOfUnits
using Printf, GeometryBasics

#### Define the Detector Parameters and Geometry
We include a file to define de structure `B2aDetector` and the construction method `B2aConstruct` to make the notebook more manageable

In [2]:
include(joinpath(@__DIR__, "DetectorB2a.jl"))

#### Define TrackerHit structure

In [3]:
struct TrackerHit
    trackID::Int32
    chamberNb::Int32
    edep::Float64
    pos::Point3{Float64}
  end
  function Base.show(io::IO, hit::TrackerHit)
    (;trackID, chamberNb, edep, pos) = hit
    @printf(io, "\ntrackID: %3d chamberNb: %2d Edep: %.3f MeV Position: (%3f, %3f, %3f)", trackID, chamberNb, edep/MeV, pos...) 
  end

#### Define Sensitive Detector for the chambers
It implies to define a data structure where to accumulate the TrackerHits for each event, and the callback functions:
- `intilialize` called at the beginning of each event
- `endOfEvent` called at the end of the event
- `processHit`  called for each hit (step ending in the associated logical volume) to be processed

all of the callback functions get the declared data structure as first argument 

In [4]:
#---SD collected data------------------------------------------------------------------------------
struct B2aSDData <: G4JLSDData
    trackerHits::Vector{TrackerHit}
    B2aSDData() = new([])
end
#---Initialize method------------------------------------------------------------------------------
function _initialize(data::B2aSDData, ::G4HCofThisEvent)::Nothing
    empty!(data.trackerHits)
    return
end
#---End of Event method----------------------------------------------------------------------------
function _endOfEvent(data::B2aSDData, ::G4HCofThisEvent)::Nothing
    return
end
#---Process Hit method-----------------------------------------------------------------------------
function _processHits(data::B2aSDData, step::G4Step, ::G4TouchableHistory)::Bool
    edep = step |> GetTotalEnergyDeposit
    edep <  0. && return false
    pos = step |> GetPostStepPoint |> GetPosition
    push!(data.trackerHits, TrackerHit(step |> GetTrack |> GetTrackID,
                                       step |> GetPreStepPoint |> GetTouchable |> GetCopyNumber,
                                       edep,
                                       Point3{Float64}(x(pos),y(pos),z(pos))))
    return true
end

_processHits (generic function with 1 method)

And create a specialized instance of `G4JLSensitiveDetector` 

In [5]:
#---Create SD instance-----------------------------------------------------------------------------
chamber_SD = G4JLSensitiveDetector("Chamber_SD", B2aSDData();           # SD name an associated data are mandatory
                                    processhits_method=_processHits,    # process hist method (also mandatory)
                                    initialize_method=_initialize,      # intialize method
                                    endofevent_method=_endOfEvent);     # end of event method


#### Define an End Event Action
This is collect all results each event. Only printing at this time. 

In [6]:
#---End Event Action-------------------------------------------------------------------------------
function endeventaction(evt::G4Event, app::G4JLApplication)
    hits = app.sdetectors["Chamber_LV+"].data.trackerHits
    eventID = evt |> GetEventID
    if eventID < 10 || eventID % 100 == 0
      println("Event: $eventID with $(length(hits)) hits stored in this event")
    end
    return
  end

endeventaction (generic function with 1 method)

#### Define the rest of the application
This includes:
- Simulation data structure `B2SimData` that will be filled with the actions
- Initialize particle gun
- Define stepping and tracking actions in this example that do storage the step positions

In [7]:
#---Particle Gun initialization--------------------------------------------------------------------
function gun_initialize(gen::G4JLParticleGun, det::G4JLDetector)
  pg = GetGun(gen)
  SetParticleByName(pg, "proton")
  SetParticleEnergy(pg, 3GeV)
  SetParticleMomentumDirection(pg, G4ThreeVector(0,0,1))
  SetParticlePosition(pg, G4ThreeVector(0,0,-16cm))
  SetParticlePosition(pg, G4ThreeVector(0, 0, -det.worldZHalfLength))
end
Geant4.getInitializer(::G4JLParticleGun) = gun_initialize

#### Instantiate a `G4JLApplication` with all the elements
The G4RunManager is created at this moment

In [8]:
#---Create the Application-------------------------------------------------------------------------
app = G4JLApplication( detector = B2aDetector(nChambers=5),          # detector with parameters
                       runmanager_type = G4RunManager,               # what RunManager to instantiate
                       physics_type = FTFP_BERT,                     # what physics list to instantiate
                       generator_type = G4JLParticleGun,             # what primary generator to instantiate
                       endeventaction_method = endeventaction,       # end event action
                       sdetectors = ["Chamber_LV+" => chamber_SD] # mapping of LVs to SDs (+ means multiple LVs with same name)
                      );             


**************************************************************
 Geant4 version Name: geant4-11-01-patch-01 [MT]   (10-February-2023)
                       Copyright : Geant4 Collaboration
                      References : NIM A 506 (2003), 250-303
                                 : IEEE-TNS 53 (2006), 270-278
                                 : NIM A 835 (2016), 186-225
                             WWW : http://geant4.org/
**************************************************************



#### Final steps
Configure the application, initialize the run and fire the beam for 0 events to finalize all initializations

In [9]:
configure(app)

<<< Geant4 Physics List simulation engine: FTFP_BERT



In [10]:
initialize(app)

Checking overlaps for volume Target:0 (G4Tubs) ... OK! 
Checking overlaps for volume Tracker:0 (G4Tubs) ... OK! 
Checking overlaps for volume Chamber_PV:1 (G4Tubs) ... OK! 
Checking overlaps for volume Chamber_PV:2 (G4Tubs) ... OK! 
Checking overlaps for volume Chamber_PV:3 (G4Tubs) ... OK! 
Checking overlaps for volume Chamber_PV:4 (G4Tubs) ... OK! 
Checking overlaps for volume Chamber_PV:5 (G4Tubs) ... OK! 

 hInelastic FTFP_BERT : threshold between BERT and FTFP is over the interval 
 for pions :   3 to 6 GeV
 for kaons :   3 to 6 GeV
 for proton :  3 to 6 GeV
 for neutron : 3 to 6 GeV

### Adding tracking cuts for neutron  TimeCut(ns)= 10000  KinEnergyCut(MeV)= 0


In [11]:
beamOn(app, 0)

======                 Electromagnetic Physics Parameters      ========
LPM effect enabled                                 1
Enable creation and use of sampling tables         0
Apply cuts on all EM processes                     0
Use combined TransportationWithMsc                 Disabled
Use general process                                1
Enable linear polarisation for gamma               0
Enable photoeffect sampling below K-shell          1
Enable sampling of quantum entanglement            0
X-section factor for integral approach             0.8
Min kinetic energy for tables                      100 eV 
Max kinetic energy for tables                      100 TeV
Number of bins per decade of a table               7
Verbose level                                      1
Verbose level for worker thread                    0
Bremsstrahlung energy threshold above which 
  primary e+- is added to the list of secondary    100 TeV
Bremsstrahlung energy threshold above which primary
  muon/ha

In [12]:
beamOn(app,10)

Event: 0 with 53 hits stored in this event
Event: 1 with 164 hits stored in this event
Event: 2 with 109 hits stored in this event
Event: 3 with 203 hits stored in this event
Event: 4 with 244 hits stored in this event
Event: 5 with 171 hits stored in this event
Event: 6 with 99 hits stored in this event
Event: 7 with 58 hits stored in this event
Event: 8 with 122 hits stored in this event
Event: 9 with 32 hits stored in this event


In [14]:
@time beamOn(app,1000)

Event: 0 with 74 hits stored in this event
Event: 1 with 633 hits stored in this event
Event: 2 with 79 hits stored in this event
Event: 3 with 106 hits stored in this event
Event: 4 with 112 hits stored in this event
Event: 5 with 137 hits stored in this event
Event: 6 with 215 hits stored in this event
Event: 7 with 81 hits stored in this event
Event: 8 with 87 hits stored in this event
Event: 9 with 103 hits stored in this event
Event: 100 with 95 hits stored in this event
Event: 200 with 408 hits stored in this event
Event: 300 with 104 hits stored in this event
Event: 400 with 87 hits stored in this event
Event: 500 with 109 hits stored in this event
Event: 600 with 153 hits stored in this event
Event: 700 with 145 hits stored in this event
Event: 800 with 288 hits stored in this event
Event: 900 with 81 hits stored in this event
  1.742872 seconds (3.55 k allocations: 1.691 MiB)
